In [25]:
import json
import pandas as pd
from collections import defaultdict
from jinja2 import Environment, FileSystemLoader

with open("./ggplot vs Python Plotting.ipynb", 'r') as f:
    nb = json.load(f)
cells = nb['cells']
tags = {i: set(c['metadata'].get('tags') or {}) for i, c in enumerate(cells)}

meta = defaultdict(list)
for cell_num, tags in tags.items():
    if 'ex' not in tags: continue
    tags = {t.split(":")[0]: t.split(":")[1] for t in tags if ":" in t}
    print(tags['name'])
    meta[tags['name']].append({
        "cell_num": cell_num,
        "package": tags["package"],
    })

def get_source_and_image(num, cells):
    image = cells[num]['outputs'][0]['data']['image/png'].replace("\n", "").strip()
    source = "".join(cells[num]['source']).replace("%%R", "").strip()
    return source, image


output = [
    ["Bar Plot of Counts", "ggplot", *get_source_and_image(6, cells)],
    ["Bar Plot of Counts", "Pandas", *get_source_and_image(7, cells)]
]


env = Environment(loader=FileSystemLoader('web'), extensions=['jinja2_highlight.HighlightExtension'])
template = env.get_template('t_index.html')
output_from_parsed_template = template.render(foo=output)


# to save the results
with open("web/index.html", "w") as fh:
    fh.write(output_from_parsed_template)

!make s3_upload

bar-counts
bar-counts
cd
s3cmd sync web/ s3://catskill-sitcom-mew --acl-public --delete-removed --guess-mime-type --no-mime-magic --no-preserve
upload: 'web/index.html' -> 's3://catskill-sitcom-mew/index.html'  [1 of 2]
 40887 of 40887   100% in    0s   646.36 kB/s  done
upload: 'web/t_index.html' -> 's3://catskill-sitcom-mew/t_index.html'  [2 of 2]
 4406 of 4406   100% in    0s   140.70 kB/s  done
Done. Uploaded 45293 bytes in 1.0 seconds, 44.23 kB/s.
